In [24]:
import requests
import sqlite3
from sqlite3 import Connection, Cursor
from requests import Response
from bs4 import BeautifulSoup
from bs4 import ResultSet
from typing import List, Tuple

In [25]:
def get_tournament_urls(page_number: int) -> List[Tuple[str]]:
    url: str = 'https://www.chess.com/club/live-tournaments/uschess-members-only?&page=' + str(page_number)
    response: Response = requests.get(url)
    soup: BeautifulSoup = BeautifulSoup(response.text)
    results: ResultSet = soup.find_all('a', {'class': 'tournaments-live-name'})
    urls: List[str] = []
    for r in results:
        if r['href']:
            urls.append((r['href'],))
    return urls

In [26]:
def create_url_table(conn: Connection) -> None:
    cur: Cursor = conn.cursor()
    query: str = """CREATE TABLE IF NOT EXISTS chess_urls (
                        id INTEGER PRIMARY KEY,
                        url TEXT
                    )"""
    cur.execute(query)
    conn.commit()

def store_urls(conn: Connection, urls: List[Tuple[str]]) -> None:
    cur: Cursor = conn.cursor()
    query: str = "INSERT INTO chess_urls (url) VALUES (?)"
    cur.executemany(query, urls)
    conn.commit()


In [27]:
# Used this code to get list of chess tournament URLs

# conn: Connection = sqlite3.Connection('scrape_data.db')
# # create_url_table(conn)

# CHESS_PAGE_COUNT = 106
# for i in range(1, CHESS_PAGE_COUNT + 1):
#     urls: List[Tuple[str]] = get_tournament_urls(i)
#     store_urls(conn, urls)


In [28]:
conn: Connection = sqlite3.Connection('scrape_data.db')
cur: Cursor = conn.cursor()
cur.execute('SELECT url from chess_urls LIMIT 1')
print(cur.fetchall())

[('https://www.chess.com/tournament/live/us-chess-blitz-u1450-3807319',)]


In [29]:
url: str = 'https://www.chess.com/tournament/live/us-chess-blitz-u1450-3807319'
response: Response = requests.get(url)
soup: BeautifulSoup = BeautifulSoup(response.text)
results: ResultSet = soup.find_all('a', {'v-tooltip': True})
count = 0
for tag in results:
    print(tag['v-tooltip'])
    count += 1
print(count)



raybo44 (#18)   vs DarkMatterGM (#1) 
DarkMatterGM (#1)   vs austonshafer (#7) 
adam54880 (#6)   vs DarkMatterGM (#1) 
DarkMatterGM (#1)   vs rexh17 (#9) 
reachvincj (#2)   vs DarkMatterGM (#1) 
DarkMatterGM (#1)   vs bdftesla (#5) 
DarkMatterGM (#1)   vs jonahtetz1 (#8) 
reachvincj (#2)   vs JadenD2 (-) 
chessbishop21 (#10)   vs reachvincj (#2) 
reachvincj (#2)   vs MarcusSuzuki90293 (-) 
Slav1265 (#11)   vs reachvincj (#2) 
reachvincj (#2)   vs DarkMatterGM (#1) 
reachvincj (#2)   vs ScotchWhiskey (#3) 
bdftesla (#5)   vs reachvincj (#2) 
ScotchWhiskey (#3)   vs acirillo (#19) 
zevulous (#12)   vs ScotchWhiskey (#3) 
ScotchWhiskey (#3)   vs rexh17 (#9) 
Sarcasma (-)   vs ScotchWhiskey (#3) 
ScotchWhiskey (#3)   vs adam54880 (#6) 
reachvincj (#2)   vs ScotchWhiskey (#3) 
austonshafer (#7)   vs ScotchWhiskey (#3) 
Levi27one (#20)   vs A_Romanov (#4) 
A_Romanov (#4)   vs ShawnChess5 (-) 
superabhi2015 (#26)   vs A_Romanov (#4) 
A_Romanov (#4)   vs johnny1668 (#15) 
acirillo (#19)   vs A

In [49]:
# Create a round number feature, that counts the number of rounds in every tournaments. 

# conn: Connection = sqlite3.Connection('scrape_data.db')
# cur: Cursor = conn.cursor()

# add_column = """ALTER TABLE uscf_tournaments
#                 ADD COLUMN rounds INTEGER"""
# cur.execute(add_column)
# conn.commit()

# query = """
#     SELECT ut.id, MAX(ur.round_number)
#     FROM uscf_rounds ur JOIN uscf_tournaments ut ON ur.uscf_tournaments_id = ut.id
#     GROUP BY ut.id
# """
# cur.execute(query)
# id_and_rounds: List[Tuple[int]] = cur.fetchall()

# add_rounds = """
# UPDATE uscf_tournaments
# SET rounds = ?
# WHERE id = ?
# """

# id_and_rounds_reversed: List[Tuple[int]] = []
# for element in id_and_rounds:
#     id_and_rounds_reversed.append(tuple(reversed(element)))

# cur.executemany(add_rounds, id_and_rounds_reversed)
# conn.commit()

In [32]:
test:str = """
raybo44 (#18)   vs DarkMatterGM (#1) 
DarkMatterGM (#1)   vs austonshafer (#7) 
adam54880 (#6)   vs DarkMatterGM (#1) 
DarkMatterGM (#1)   vs rexh17 (#9) 
reachvincj (#2)   vs DarkMatterGM (#1) 
DarkMatterGM (#1)   vs bdftesla (#5) 
DarkMatterGM (#1)   vs jonahtetz1 (#8) 
reachvincj (#2)   vs JadenD2 (-) 
chessbishop21 (#10)   vs reachvincj (#2) 
reachvincj (#2)   vs MarcusSuzuki90293 (-) 
Slav1265 (#11)   vs reachvincj (#2) 
reachvincj (#2)   vs DarkMatterGM (#1) 
reachvincj (#2)   vs ScotchWhiskey (#3) 
bdftesla (#5)   vs reachvincj (#2) 
ScotchWhiskey (#3)   vs acirillo (#19) 
zevulous (#12)   vs ScotchWhiskey (#3) 
ScotchWhiskey (#3)   vs rexh17 (#9) 
Sarcasma (-)   vs ScotchWhiskey (#3) 
ScotchWhiskey (#3)   vs adam54880 (#6) 
reachvincj (#2)   vs ScotchWhiskey (#3) 
austonshafer (#7)   vs ScotchWhiskey (#3) 
Levi27one (#20)   vs A_Romanov (#4) 
A_Romanov (#4)   vs ShawnChess5 (-) 
superabhi2015 (#26)   vs A_Romanov (#4) 
A_Romanov (#4)   vs johnny1668 (#15) 
acirillo (#19)   vs A_Romanov (#4) 
A_Romanov (#4)   vs Slav1265 (#11) 
A_Romanov (#4)   vs adam54880 (#6) 
bdftesla (#5)   vs badWolfe42 (-) 
MarcusSuzuki90293 (-)   vs bdftesla (#5) 
bdftesla (#5)   vs chessbishop21 (#10) 
austonshafer (#7)   vs bdftesla (#5) 
bdftesla (#5)   vs Slav1265 (#11) 
DarkMatterGM (#1)   vs bdftesla (#5) 
bdftesla (#5)   vs reachvincj (#2) 
Jazzedler (#17)   vs adam54880 (#6) 
jonahtetz1 (#8)   vs adam54880 (#6) 
adam54880 (#6)   vs DarkMatterGM (#1) 
adam54880 (#6)   vs ShawnChess5 (-) 
ScotchWhiskey (#3)   vs adam54880 (#6) 
adam54880 (#6)   vs chessbishop21 (#10) 
A_Romanov (#4)   vs adam54880 (#6) 
austonshafer (#7)   vs KrithikSathiya21 (#25) 
DarkMatterGM (#1)   vs austonshafer (#7) 
austonshafer (#7)   vs raybo44 (#18) 
austonshafer (#7)   vs bdftesla (#5) 
MarcusSuzuki90293 (-)   vs austonshafer (#7) 
ffmedic11 (#13)   vs austonshafer (#7) 
austonshafer (#7)   vs ScotchWhiskey (#3) 
Sarcasma (-)   vs jonahtetz1 (#8) 
jonahtetz1 (#8)   vs adam54880 (#6) 
Slav1265 (#11)   vs jonahtetz1 (#8) 
jonahtetz1 (#8)   vs Corzican (#16) 
rexh17 (#9)   vs jonahtetz1 (#8) 
jonahtetz1 (#8)   vs MarcusSuzuki90293 (-) 
DarkMatterGM (#1)   vs jonahtetz1 (#8) 
ShawnChess5 (-)   vs rexh17 (#9) 
rexh17 (#9)   vs Levi27one (#20) 
ScotchWhiskey (#3)   vs rexh17 (#9) 
DarkMatterGM (#1)   vs rexh17 (#9) 
rexh17 (#9)   vs jonahtetz1 (#8) 
zevulous (#12)   vs rexh17 (#9) 
rexh17 (#9)   vs Corzican (#16) 
chessbishop21 (#10)   vs ffmedic11 (#13) 
chessbishop21 (#10)   vs reachvincj (#2) 
bdftesla (#5)   vs chessbishop21 (#10) 
chessbishop21 (#10)   vs KrithikSathiya21 (#25) 
c4luke1 (#14)   vs chessbishop21 (#10) 
adam54880 (#6)   vs chessbishop21 (#10) 
chessbishop21 (#10)   vs raybo44 (#18) 
Slav1265 (#11)   vs GMSumeetle (#21) 
Penguin_Michael (#27)   vs Slav1265 (#11) 
Slav1265 (#11)   vs jonahtetz1 (#8) 
Slav1265 (#11)   vs reachvincj (#2) 
bdftesla (#5)   vs Slav1265 (#11) 
A_Romanov (#4)   vs Slav1265 (#11) 
Slav1265 (#11)   vs johnny1668 (#15) 
johnny1668 (#15)   vs zevulous (#12) 
zevulous (#12)   vs ScotchWhiskey (#3) 
ShawnChess5 (-)   vs zevulous (#12) 
zevulous (#12)   vs GMSumeetle (#21) 
Corzican (#16)   vs zevulous (#12) 
zevulous (#12)   vs rexh17 (#9) 
zevulous (#12)   vs ffmedic11 (#13) 
chessbishop21 (#10)   vs ffmedic11 (#13) 
ffmedic11 (#13)   vs JadenD2 (-) 
acirillo (#19)   vs ffmedic11 (#13) 
ffmedic11 (#13)   vs MarcusSuzuki90293 (-) 
Sarcasma (-)   vs ffmedic11 (#13) 
ffmedic11 (#13)   vs austonshafer (#7) 
zevulous (#12)   vs ffmedic11 (#13) 
Jazzedler (#17)   vs c4luke1 (#14) 
raybo44 (#18)   vs c4luke1 (#14) 
c4luke1 (#14)   vs chessbishop21 (#10) 
c4luke1 (#14)   vs bouRN84 (#22) 
Crusaders1096 (#23)   vs c4luke1 (#14) 
johnny1668 (#15)   vs zevulous (#12) 
johnny1668 (#15)   vs raybo44 (#18) 
A_Romanov (#4)   vs johnny1668 (#15) 
Jazzedler (#17)   vs johnny1668 (#15) 
johnny1668 (#15)   vs acirillo (#19) 
Slav1265 (#11)   vs johnny1668 (#15) 
Corzican (#16)   vs Jazzedler (#17) 
Levi27one (#20)   vs Corzican (#16) 
jonahtetz1 (#8)   vs Corzican (#16) 
Corzican (#16)   vs zevulous (#12) 
raybo44 (#18)   vs Corzican (#16) 
rexh17 (#9)   vs Corzican (#16) 
Jazzedler (#17)   vs adam54880 (#6) 
Corzican (#16)   vs Jazzedler (#17) 
Jazzedler (#17)   vs c4luke1 (#14) 
superabhi2015 (#26)   vs Jazzedler (#17) 
Jazzedler (#17)   vs johnny1668 (#15) 
Levi27one (#20)   vs Jazzedler (#17) 
GMSumeetle (#21)   vs Jazzedler (#17) 
raybo44 (#18)   vs DarkMatterGM (#1) 
johnny1668 (#15)   vs raybo44 (#18) 
austonshafer (#7)   vs raybo44 (#18) 
raybo44 (#18)   vs c4luke1 (#14) 
GMSumeetle (#21)   vs raybo44 (#18) 
raybo44 (#18)   vs Corzican (#16) 
chessbishop21 (#10)   vs raybo44 (#18) 
ScotchWhiskey (#3)   vs acirillo (#19) 
acirillo (#19)   vs bouRN84 (#22) 
acirillo (#19)   vs ffmedic11 (#13) 
Levi27one (#20)   vs acirillo (#19) 
acirillo (#19)   vs A_Romanov (#4) 
johnny1668 (#15)   vs acirillo (#19) 
Levi27one (#20)   vs A_Romanov (#4) 
rexh17 (#9)   vs Levi27one (#20) 
Levi27one (#20)   vs Corzican (#16) 
Levi27one (#20)   vs acirillo (#19) 
bouRN84 (#22)   vs Levi27one (#20) 
Levi27one (#20)   vs Jazzedler (#17) 
Levi27one (#20)   vs KrithikSathiya21 (#25) 
Slav1265 (#11)   vs GMSumeetle (#21) 
GMSumeetle (#21)   vs Sarcasma (-) 
bouRN84 (#22)   vs GMSumeetle (#21) 
zevulous (#12)   vs GMSumeetle (#21) 
GMSumeetle (#21)   vs raybo44 (#18) 
KrithikSathiya21 (#25)   vs GMSumeetle (#21) 
GMSumeetle (#21)   vs Jazzedler (#17) 
acirillo (#19)   vs bouRN84 (#22) 
bouRN84 (#22)   vs GMSumeetle (#21) 
bouRN84 (#22)   vs Levi27one (#20) 
c4luke1 (#14)   vs bouRN84 (#22) 
bouRN84 (#22)   vs BraydenPlaysWithStyle (#24) 
KrithikSathiya21 (#25)   vs Crusaders1096 (#23) 
Crusaders1096 (#23)   vs superabhi2015 (#26) 
Crusaders1096 (#23)   vs c4luke1 (#14) 
bouRN84 (#22)   vs BraydenPlaysWithStyle (#24) 
austonshafer (#7)   vs KrithikSathiya21 (#25) 
KrithikSathiya21 (#25)   vs Sarcasma (-) 
chessbishop21 (#10)   vs KrithikSathiya21 (#25) 
KrithikSathiya21 (#25)   vs Crusaders1096 (#23) 
KrithikSathiya21 (#25)   vs GMSumeetle (#21) 
Levi27one (#20)   vs KrithikSathiya21 (#25) 
"""

In [33]:
test_list: List[str] = test.split('\n')
formatted = []
for line in test_list:
    pairing = line.strip().split('vs')
    formatted.append(tuple(pairing))

In [34]:
import pandas as pd
array = []
for row in formatted:
    if len(row) > 1:
        white = row[0]
        black = row[1]
        white = white.strip()
        black = black.strip()
        white_split = white.split(' ')
        black_split = black.split(' ')
        white_user = white_split[0]
        white_seed = white_split[1]
        black_user = black_split[0]
        black_seed = black_split[1]
        white_seed= white_seed.strip('(')
        white_seed=white_seed.strip(')')
        white_seed=white_seed.strip('#')
        black_seed=black_seed.strip('(')
        black_seed=black_seed.strip(')')
        black_seed=black_seed.strip('#')
        array.append((white_user, white_seed, black_user, black_seed))
df = pd.DataFrame(array, columns=['white', 'white_seed', 'black', 'black_seed'])

In [35]:
slice = df[['white_seed', 'black_seed']].sort_values(by='white_seed')
slice

,white_seed,black_seed
17,-,3
29,-,5
46,-,7
49,-,8
79,-,12
...,...,...
60,9,8
53,9,8
129,9,20
57,9,20


In [36]:
conn: Connection = sqlite3.Connection('scrape_data.db')
cur: Cursor = conn.cursor()
cur.execute("""
    SELECT seed_number, opponent
    FROM uscf_rounds ur JOIN uscf_player_observations upo 
        ON ur.uscf_player_id = upo.id
        JOIN uscf_tournaments ut ON ut.id = ur.uscf_tournaments_id 
    WHERE 7 =
""")
cur.execute("SELECT section_count FROM uscf_tournaments")
result = cur.fetchall()
result = sorted(result, key=lambda x: x[0])
for r in result:
    print(r)

OperationalError: incomplete input